In [ ]:
# Real-Time Sentiment Analysis: Advanced NLP for User Feedback
# This notebook implements a state-of-the-art pipeline for real-time sentiment analysis of user input.
# It uses a pre-trained BERT model for accurate sentiment classification, interactive Plotly visualizations,
# and a Streamlit interface for deployment, optimized for business stakeholders and portfolio presentations.

# Importing core libraries for NLP, visualization, and data handling
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from transformers import pipeline
import streamlit as st
from datetime import datetime
import os
import warnings
warnings.filterwarnings('ignore')

# Setting up a professional visualization theme
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("deep")
plt.rcParams['axes.titlesize'] = 16
plt.rcParams['axes.labelsize'] = 12
plt.rcParams['font.family'] = 'Arial'

# --- 1. Sentiment Analysis Setup ---
# Initialize BERT-based sentiment analysis pipeline
sentiment_analyzer = pipeline('sentiment-analysis', model='distilbert-base-uncased-finetuned-sst-2-english')

# Initialize data storage for user inputs
if not os.path.exists('user_feedback.csv'):
    pd.DataFrame(columns=['Timestamp', 'Input', 'Sentiment', 'Score']).to_csv('user_feedback.csv', index=False)

# --- 2. Real-Time Sentiment Analysis ---
def analyze_sentiment(text):
    """Analyze sentiment of user input using BERT model."""
    if not text.strip():
        return 'Neutral', 0.0
    result = sentiment_analyzer(text)[0]
    label = result['label'].capitalize()
    score = result['score']
    return label, score

# Streamlit interface
st.title('Real-Time Sentiment Analysis')
st.markdown('Enter feedback to analyze sentiment in real-time. Results are visualized and stored for analysis.')

user_input = st.text_input('Please Rate Our Services:', '')
if user_input:
    sentiment, score = analyze_sentiment(user_input)
    st.write(f'Sentiment: **{sentiment}** (Confidence: {score:.2%})')
    
    # Save input to CSV
    feedback_df = pd.DataFrame({
        'Timestamp': [datetime.now().strftime('%Y-%m-%d %H:%M:%S')],
        'Input': [user_input],
        'Sentiment': [sentiment],
        'Score': [score]
    })
    feedback_df.to_csv('user_feedback.csv', mode='a', header=False, index=False)

# --- 3. Feedback Visualization ---
# Load stored feedback
feedback_data = pd.read_csv('user_feedback.csv')

# Sentiment distribution
if not feedback_data.empty:
    sentiment_counts = feedback_data['Sentiment'].value_counts()
    fig1 = px.bar(x=sentiment_counts.index, y=sentiment_counts.values,
                  title='Sentiment Distribution of User Feedback',
                  labels={'x': 'Sentiment', 'y': 'Number of Inputs'},
                  color=sentiment_counts.index, color_discrete_sequence=px.colors.qualitative.Set2)
    fig1.update_layout(height=500, title_x=0.5, showlegend=False)
    st.plotly_chart(fig1)

    # Sentiment score over time
    fig2 = px.scatter(feedback_data, x='Timestamp', y='Score', color='Sentiment',
                      title='Sentiment Scores Over Time',
                      labels={'Score': 'Confidence Score', 'Timestamp': 'Time'},
                      color_discrete_sequence=px.colors.qualitative.Pastel)
    fig2.update_layout(height=500, title_x=0.5)
    st.plotly_chart(fig2)

# --- 4. Business Insights ---
if not feedback_data.empty:
    st.subheader('Business Insights')
    pos_rate = (feedback_data['Sentiment'] == 'Positive').mean() * 100
    neg_rate = (feedback_data['Sentiment'] == 'Negative').mean() * 100
    st.write(f"1. Positive Feedback Rate: {pos_rate:.1f}%")
    st.write(f"2. Negative Feedback Rate: {neg_rate:.1f}%")
    st.write("3. Action Plan: Use positive feedback to highlight strengths; address negative feedback for service improvements.")
    st.write("4. Model Strength: BERT-based model ensures accurate sentiment classification.")
    st.write("5. Future Work: Integrate multilingual support or topic modeling for deeper insights.")

# --- 5. Output Preservation ---
# Save visualizations
if not feedback_data.empty:
    fig1.write_html('sentiment_distribution.html')
    fig2.write_html('sentiment_over_time.html')
    st.write("Interactive visualizations saved as 'sentiment_distribution.html' and 'sentiment_over_time.html'.")